In [1]:
import pandas as pd
import numpy as np
import json
import os
import capstone_functions as cf
import capstone_json_functions as cjf
import sql_functions as sf
from pathlib import Path
import re


In [2]:
directory = "data/kickstarter_json/"
filename = "2022-06-09.json"
file_path = f"{directory}{filename}"
schema = 'kickstarter_bg'
table_name = 'boardgames'
engine = sf.get_engine()


Rename files, if they start with "Kickstarter..."

In [ ]:
cjf.rename_kickstarter_files(directory)

Read the file and store it into a list called "lines"

In [ ]:
# lines is a list of strings
lines = cjf.list_of_lines(file_path)

build a dictionairy by looking at the "category"

In [ ]:
list_of_categories = cjf.list_of_categories(file_path)

Build the dataframe

In [ ]:
df = cjf.build_df_from_lines_list(lines)

List of Columns to keep

In [14]:
columns_to_keep = cjf.columns_to_keep

In [17]:
columns_to_keep[0] = 'game_name'

In [18]:
columns_to_keep

['game_name',
 'goal',
 'pledged',
 'state',
 'slug',
 'country',
 'currency',
 'deadline',
 'created_at',
 'launched_at',
 'backers_count',
 'usd_pledged',
 'creator_name',
 'state_changed_at']

Overwrite the df with itself with the chosen columns

In [ ]:
df = df[columns_to_keep]

Convert Unix timestamp to pandas datetime

In [ ]:
df = cjf.unix_to_datetime(df)

# And now with all the json files

In [3]:
pathlist = Path(directory).glob('**/*.json')
df_all = pd.DataFrame()
counter = 0
for path in pathlist:
    file_path = str(path)
    print(file_path)
    lines = cjf.list_of_lines(file_path)
    df_temp = cjf.build_df_from_lines_list(lines)
    df_all = pd.concat([df_all, df_temp])
    counter += 1
print(f"{counter} files have been dataframed")


data/kickstarter_json/2019-04-18.json
data/kickstarter_json/2018-09-13.json
data/kickstarter_json/2017-09-15.json
data/kickstarter_json/2021-05-13.json
data/kickstarter_json/2018-03-15.json
data/kickstarter_json/2016-06-15.json
data/kickstarter_json/2019-06-13.json
data/kickstarter_json/2019-01-17.json
data/kickstarter_json/2017-10-15.json
data/kickstarter_json/2020-10-15.json
data/kickstarter_json/2018-01-12.json
data/kickstarter_json/2021-07-15.json
data/kickstarter_json/2020-03-12.json
data/kickstarter_json/2016-07-15.json
data/kickstarter_json/2018-02-15.json
data/kickstarter_json/2020-04-16.json
data/kickstarter_json/2019-10-17.json
data/kickstarter_json/2020-02-13.json
data/kickstarter_json/2017-11-15.json
data/kickstarter_json/2017-01-15.json
data/kickstarter_json/2017-08-15.json
data/kickstarter_json/2020-11-12.json
data/kickstarter_json/2016-05-15.json
data/kickstarter_json/2021-02-11.json
data/kickstarter_json/2017-03-15.json
data/kickstarter_json/2022-04-21.json
data/kicksta

In [19]:
df_all = df_all[columns_to_keep]

In [20]:
df_all = cjf.unix_to_datetime(df_all)

/Users/ruperthufnagel/DailyLama/Daily-Lama-Capstone/capstone_json_functions.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col],unit='s')
/Users/ruperthufnagel/DailyLama/Daily-Lama-Capstone/capstone_json_functions.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col],unit='s')
/Users/ruperthufnagel/DailyLama/Daily-Lama-Capstone/capstone_json_functions.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [5]:
df_all.rename({"name":"game_name"},axis=1,inplace=True)

In [21]:
df2 = df_all.astype({'usd_pledged': f"{'float64'}"}).round(2)

In [22]:
df2['game_name'] =  df2['game_name'].str.findall(r'\w|\s').str.join('').str.replace(r"\s+","_").str.lower()

/var/folders/kb/wqf78x2d06z43w0nwzbclk3m0000gn/T/ipykernel_4084/3414864267.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['game_name'] =  df2['game_name'].str.findall(r'\w|\s').str.join('').str.replace(r"\s+","_").str.lower()


In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260659 entries, 0 to 0
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   game_name         260659 non-null  object        
 1   goal              260659 non-null  int64         
 2   pledged           260659 non-null  float64       
 3   state             260659 non-null  object        
 4   slug              260659 non-null  object        
 5   country           260659 non-null  object        
 6   currency          260659 non-null  object        
 7   deadline          260659 non-null  datetime64[ns]
 8   created_at        260659 non-null  datetime64[ns]
 9   launched_at       260659 non-null  datetime64[ns]
 10  backers_count     260659 non-null  int64         
 11  usd_pledged       260659 non-null  float64       
 12  creator_name      260659 non-null  object        
 13  state_changed_at  260659 non-null  datetime64[ns]
dtypes: dateti

In [ ]:
df2.drop_duplicates(inplace=True)

In [1]:
schema = 'bgg_data'
main_table_name = 'main'
name_table = 'game_names'
kickstarter_table = 'kickstarter_temp'

In [26]:
main_query = f"SELECT * FROM {schema}.{main_table_name};"
name_query = f"SELECT * FROM {schema}.{name_table};"
kickstarter_query = f"SELECT * FROM {schema}.{kickstarter_temp};"

In [27]:
df_main_bgg = sf.get_dataframe(main_query)
df_name_bgg = sf.get_dataframe(name_query)

In [42]:
df_main_bgg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137153 entries, 0 to 137152
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   id             137153 non-null  int64
 1   yearpublished  137153 non-null  int64
 2   min_players    137153 non-null  int64
 3   max_players    137153 non-null  int64
 4   playtime       137153 non-null  int64
 5   min_playtime   137153 non-null  int64
 6   max_playtime   137153 non-null  int64
 7   min_age        137153 non-null  int64
dtypes: int64(8)
memory usage: 8.4 MB


In [28]:
df_name_main_bgg = pd.merge(df_main_bgg,df_name_bgg,how='inner',on='id')

In [30]:
df_name_main_bgg = cf.rename_game_names(df_name_main_bgg,column_name='game_name')

/Users/ruperthufnagel/DailyLama/Daily-Lama-Capstone/capstone_functions.py:128: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column_name] =  df[column_name].str.findall(r'\w|\s').str.join('').str.replace(r"\s+","_").str.lower()


In [41]:
df_name_main_bgg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206097 entries, 0 to 206096
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             206097 non-null  int64 
 1   yearpublished  206097 non-null  int64 
 2   min_players    206097 non-null  int64 
 3   max_players    206097 non-null  int64 
 4   playtime       206097 non-null  int64 
 5   min_playtime   206097 non-null  int64 
 6   max_playtime   206097 non-null  int64 
 7   min_age        206097 non-null  int64 
 8   game_name      206096 non-null  object
 9   primary_bool   206097 non-null  int64 
dtypes: int64(9), object(1)
memory usage: 17.3+ MB


In [43]:
df2.drop_duplicates(inplace=True)

,id,yearpublished,min_players,max_players,playtime,min_playtime,max_playtime,min_age,game_name,primary_bool,...,slug,country,currency,deadline,created_at,launched_at,backers_count,usd_pledged,creator_name,state_changed_at
0,98443,2012,1,4,90,90,90,10,dark_horse,1,...,dark-horse,US,USD,2011-06-21 19:49:09,2011-04-19 01:36:55,2011-05-07 19:49:09,148,9696.03,Knight Works,2011-06-21 19:49:10
1,98443,2012,1,4,90,90,90,10,dark_horse,1,...,dark-horse,US,USD,2011-06-21 19:49:09,2011-04-19 01:36:55,2011-05-07 19:49:09,148,9696.03,Knight Works,2011-06-21 19:49:10
2,98443,2012,1,4,90,90,90,10,dark_horse,1,...,dark-horse,US,USD,2011-06-21 19:49:09,2011-04-19 01:36:55,2011-05-07 19:49:09,148,9696.03,Knight Works,2011-06-21 19:49:10
3,98443,2012,1,4,90,90,90,10,dark_horse,1,...,dark-horse,US,USD,2011-06-21 19:49:09,2011-04-19 01:36:55,2011-05-07 19:49:09,148,9696.03,Knight Works,2011-06-21 19:49:10
4,98443,2012,1,4,90,90,90,10,dark_horse,1,...,dark-horse,US,USD,2011-06-21 19:49:09,2011-04-19 01:36:55,2011-05-07 19:49:09,148,9696.03,Knight Works,2011-06-21 19:49:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,173093,2016,2,4,120,60,120,14,warquest,1,...,warquest,US,USD,2015-05-28 22:00:00,2015-04-22 23:48:42,2015-05-05 13:53:03,363,84506.00,Mr. B. Games,2015-05-28 22:00:12
196,173093,2016,2,4,120,60,120,14,warquest,1,...,warquest,US,USD,2015-05-28 22:00:00,2015-04-22 23:48:42,2015-05-05 13:53:03,363,84506.00,Mr. B. Games,2015-05-28 22:00:12
197,173093,2016,2,4,120,60,120,14,warquest,1,...,warquest,US,USD,2015-05-28 22:00:00,2015-04-22 23:48:42,2015-05-05 13:53:03,363,84506.00,Mr. B. Games,2015-05-28 22:00:12
198,173093,2016,2,4,120,60,120,14,warquest,1,...,warquest,US,USD,2015-05-28 22:00:00,2015-04-22 23:48:42,2015-05-05 13:53:03,363,84506.00,Mr. B. Games,2015-05-28 22:00:12


In [32]:
df2["bgg_game_name_id"] = df_name_main_bgg["id"].loc[df_name_main_bgg["game_name"] == df2["game_name"]]

ValueError: Can only compare identically-labeled Series objects

In [44]:
tablename = 'kickstarter_temp'
sf.build_table(engine, tablename, df2, schema)


The kickstarter_temp table was imported successfully.
